In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO
import os
import sys
import math
from tdc.multi_pred import PeptideMHC

install tdc: https://tdc.readthedocs.io/en/main/install.html

### Data from NetMHCII32

We used training and test data from NetMHCIIpan 3.2 (https://services.healthtech.dtu.dk/service.php?NetMHCIIpan-3.2) and mhc2_iedb_jensen (https://tdcommons.ai/multi_pred_tasks/peptidemhc/) to match the pseudosequence of MHCII.

In [20]:
data = PeptideMHC(name = 'MHC2_IEDB_Jensen')
split = data.get_split()

Downloading...
100%|██████████| 11.8M/11.8M [00:01<00:00, 8.01MiB/s]
Loading...
Done!


In [21]:
jensen = pd.read_csv(os.path.join(sys.path[0],'data/mhc2_iedb_jensen.tab'), sep="\t")

In [22]:
jensen.columns = ['Peptide_ID', 'Peptide', 'Y', 'MHC_ID', 'MHC']
jensen.head()


,Peptide_ID,Peptide,Y,MHC_ID,MHC
0,0,PKYVKQNTLKLAT,0.000000,HLA-DPA10103-DPB10201,YAFFMFSGGAILNTLFGQFEYFDIEEVRMHLGMT
1,1,DSDVGEFRAVTELG,0.047212,HLA-DPA10103-DPB10201,YAFFMFSGGAILNTLFGQFEYFDIEEVRMHLGMT
2,2,AAAAGWQTLSAALDA,0.238910,HLA-DPA10103-DPB10201,YAFFMFSGGAILNTLFGQFEYFDIEEVRMHLGMT
3,3,AALDAQAVELTARLN,0.357937,HLA-DPA10103-DPB10201,YAFFMFSGGAILNTLFGQFEYFDIEEVRMHLGMT
4,4,AAPAAGYTPATPAAP,0.076722,HLA-DPA10103-DPB10201,YAFFMFSGGAILNTLFGQFEYFDIEEVRMHLGMT


In [6]:
unique_MHC = jensen[["MHC_ID", "MHC"]].drop_duplicates(subset=["MHC_ID", "MHC"]).reset_index(drop=True)
print(unique_MHC.shape, unique_MHC.MHC.nunique())

(80, 2) 75


In [7]:
def read_dataset(fold_number):
    train1=pd.read_csv(os.path.join(sys.path[0],"NetMHCIIpan32/train%d.txt"%(fold_number)) , sep="\t", header=None, names=["Peptide", "Y", "MHC_ID"])
    test1=pd.read_csv(os.path.join(sys.path[0],"NetMHCIIpan32/test%d.txt"%(fold_number)), sep="\t", header=None, names=["Peptide", "Y", "MHC_ID"])


    train1_1 = train1[train1["MHC_ID"].isin(list(unique_MHC.MHC_ID.values))]
    train1_1 = train1_1.merge(unique_MHC, on=["MHC_ID"], how="left")
    test1_1 = test1[test1["MHC_ID"].isin(list(unique_MHC.MHC_ID.values))]
    test1_1 = test1_1.merge(unique_MHC, on=["MHC_ID"], how="left")

    print(train1_1.shape)
    print(test1_1.shape)

    return train1_1, test1_1

In [8]:
train1, test1 = read_dataset(1)

(106949, 4)
(27332, 4)


### include only human data

In [9]:
# exclude mice data
def exclude_mice(df):
    jensen2 = df[~(df["MHC_ID"].str.contains("H-2"))].reset_index(drop=True)

    print(jensen2.shape)
    print(jensen2.MHC_ID.nunique())

    return jensen2

In [10]:
train1_1 = exclude_mice(train1)
test1_1 = exclude_mice(test1)

(104266, 4)
72
(26742, 4)
65


### create outcomes

In [11]:
def create_outcome(df):
    df["y"] = (df["Y"]>0.426).astype(int) 
    print(df.y.value_counts())
    return df
# 1: binders

In [12]:
train1_1 = create_outcome(train1_1)
test1_1 = create_outcome(test1_1)

0    60127
1    44139
Name: y, dtype: int64
0    15893
1    10849
Name: y, dtype: int64


### MHC with more than 20 peptides

In [13]:
# with more than 20 peptides
def peptide20(df):
    jensen2 = df
    peptide20=[]
    for i in jensen2.MHC_ID.unique():
        df_try = jensen2[jensen2["MHC_ID"]==i]
        a = len(df_try)
        if a >20:
            peptide20.append(i)

    print(len(peptide20))

    jensen4 = jensen2.loc[jensen2["MHC_ID"].isin(peptide20)].reset_index(drop=True)
    print(jensen4.MHC_ID.nunique())
    print(jensen4.shape)

    return jensen4

In [14]:
train1_2 = peptide20(train1_1)
test1_2 = peptide20(test1_1)

58
58
(104186, 5)
50
50
(26640, 5)


### MHC with at least 4 binders

In [15]:
# at least 4 binders
## find ID with at least 4 binders

def binder4(df):
    jensen4=df
    peptide4 = []

    for i in jensen4.MHC_ID.unique():
        df_try = jensen4[jensen4["MHC_ID"]==i]
        a = df_try.y.sum()
        if a >=4:
            peptide4.append(i)

    print(len(peptide4))


    jensen5 = jensen4.loc[jensen4["MHC_ID"].isin(peptide4)]
    jensen5 = jensen5.reset_index(drop=True)
    print(jensen5.MHC_ID.nunique())
    print(jensen5.shape)
    
    return jensen5

In [16]:
train1_3 = binder4(train1_2)
test1_3 = binder4(test1_2)

53
53
(104040, 5)
50
50
(26640, 5)


### calculate peptide length and keep those of length 15

In [17]:
def peptideL(df):
    df["Peptide_L"] = df.Peptide.str.len()
    df = df[df["Peptide_L"]==15].reset_index(drop=True)

    # create Peptide_ID and Pair_ID
    df["Peptide_ID"] = list(range(0,len(df),1))
    df["Pair_ID"]= list(range(len(df),len(df)+len(df),1))
    print(df.MHC_ID.nunique())
    print(df.shape)
    return df

In [18]:
train1_4 = peptideL(train1_3)
test1_4 = peptideL(test1_3)


52
(87052, 8)
50
(21535, 8)


### fasta

In [53]:
# Separate peptide and MHC and do preprocessing separately

def to_fasta(file, filenumber):
    print("The file %s has %d number of data" %(filenumber, len(file)))

    # Peptide: drop duplicates by sequence
    data_peptide = file[["Pair_ID", "Peptide_ID", "Peptide"]].drop_duplicates(subset=["Peptide"], keep="last")
    print("unique peptide", len(data_peptide))

    # MHC: drop duplicates by MHC_ID
    data_MHC = file[["Pair_ID", "MHC_ID", "MHC"]].drop_duplicates(subset=["MHC_ID"], keep="last")
    print("unique MHC", len(data_MHC))


    ## Peptide
    
    txt_file = os.path.join(sys.path[0],'Analysis/Peptide_%s.txt' %(filenumber))
    with open(txt_file, 'w') as f:
        for i in data_peptide.index:
            
            l1 = data_peptide["Peptide_ID"][i].astype(str)
            l2 = data_peptide["Peptide"][i]
            f.write(l1)
            f.write("\t")
            f.write(l2)
            f.write("\n")
    
    count = SeqIO.convert(os.path.join(sys.path[0],"Analysis/Peptide_%s.txt" %(filenumber)), "tab", os.path.join(sys.path[0],"Analysis/Peptide_%s.fasta"%(filenumber)), "fasta")
    print("Converted %i records" % count)

    ## MHC

    txt_file = os.path.join(sys.path[0],'Analysis/MHC_%s.txt'%(filenumber))
    with open(txt_file, 'w') as f:
        for i in data_MHC.index:
            l1 = data_MHC["MHC_ID"][i]
            l2 = data_MHC["MHC"][i]

            f.write(l1)
            f.write("\t")
            f.write(l2)
            f.write("\n")


    count = SeqIO.convert(os.path.join(sys.path[0],"Analysis/MHC_%s.txt" %(filenumber)), "tab", os.path.join(sys.path[0],"Analysis/MHC_%s.fasta" %(filenumber)), "fasta")
    print("Converted %i records" % count)



In [54]:
to_fasta(train1_4, "train1_train")
to_fasta(test1_4, "train1_test")

The file train1_train has 87052 number of data
unique peptide 9611
unique MHC 52
Converted 9611 records
Converted 52 records
The file train1_test has 21535 number of data
unique peptide 2778
unique MHC 50
Converted 2778 records
Converted 50 records


In [56]:
# also save the filtered datasets
train1_4.to_csv(os.path.join(sys.path[0], "Analysis/train1_sample.csv"), index=False)
test1_4.to_csv(os.path.join(sys.path[0], "Analysis/test1_sample.csv"), index=False)